In [2]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def identity(words):
    return words

def create_pipeline(estimator, reduction=False):
    steps = [
        ('normalize', TextNormalizer()),
        ('vectorize', TfidfVectorizer(
            tokenizer=identity, preprocessor=None, lowercase=False
        ))
    ]
    if reduction:
        steps.append(('reduction', TruncatedSVD(n_components=10_000)))
        
    # Add the estimator.
    steps.append('classifier', estimator)
    return Pipeline(steps)

In [4]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB

models = []
for form in (LogisticRegression, MultinomialNB, SGDClassifier):
    models.append(create_pipeline(form(), True))
    models.append(create_pipeline(form(), False))
    
for model in models:
    model.fit(train_docs, train_labels)
    
    
for model in models:
    scores = []
    
    for X_train, X_test, y_train, y_test in loader:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        score = accuracy_score(y_test, y_pred)
        scores.append(score)
    print('Accuracy of {} is {:0.3f}'.format(model, np.mean(scores)))

NameError: name 'TextNormalizer' is not defined

In [5]:
from sklearn.metrics import classification_report

model = create_pipeline(SGDClassifier(), False)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, labels=labels))

NameError: name 'TextNormalizer' is not defined

In [ ]:
import tabulate
import numpy as np

from collections import defaultdict
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score

fields = ['model', 'precision', 'recall', 'accuracy', 'f1']
table = []

for model in models:
    scores = defaultdict(list) # Storage for all our model metrics.
    
    # K-fold cross-validation.
    for X_train, X_test, y_train, y_test in loader:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # Add scores to our scores.
        scores['precision'].append(precision_score(y_test, y_pred))
        scores['recall'].append(recall_score(y_test, y_pred))
        scores['accuracy'].append(accuracy_score(y_test, y_pred))
        scores['f1'].append(f1_score(y_test, y_pred))
    
    # Aggregate our scores and add to the table.
    row = [str(model)]
    for field in fields[1:]:
        row.append(np.mean(scores[field]))
    
    table.append(row)
    
# Sort the models by F1 score descending.
table.sort(key=lambda row: row[-1], reverse=True)
print(tabulate.tabulate(table, headers=fields))